<a href="https://colab.research.google.com/github/AlexeyTri/MLSeminars/blob/main/MLHSE/Seminar9_knn_ish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/Murcha1990/MLDS_ML_2022/main/%D0%97%D0%B0%D0%BD%D1%8F%D1%82%D0%B8%D0%B5%209_knn_lsh/Tweets.csv", delimiter=',', error_bad_lines=False)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 284: expected 4 fields, saw 5\nSkipping line 2398: expected 4 fields, saw 5\nSkipping line 2438: expected 4 fields, saw 6\nSkipping line 3104: expected 4 fields, saw 6\nSkipping line 3618: expected 4 fields, saw 5\nSkipping line 3742: expected 4 fields, saw 7\nSkipping line 4694: expected 4 fields, saw 9\nSkipping line 5272: expected 4 fields, saw 5\nSkipping line 6194: expected 4 fields, saw 5\nSkipping line 7017: expected 4 fields, saw 6\nSkipping line 7786: expected 4 fields, saw 5\nSkipping line 8407: expected 4 fields, saw 5\nSkipping line 8529: expected 4 fields, saw 5\nSkipping line 8820: expected 4 fields, saw 5\nSkipping line 9157: expected 4 fields, saw 7\nSkipping line 9952: expected 4 fields, saw 5\nSkipping line 10203: expec

In [7]:
df = df[['text']].dropna()

In [9]:
import numpy as np

In [10]:
df.reset_index(drop=True, inplace=True)
df['id'] = np.arange(len(df))

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1,3),
    min_df=0,
    stop_words='english'
)

X_tfidf = tfidf.fit_transform(df['text'])
X_tfidf

<21027x24002 sparse matrix of type '<class 'numpy.float64'>'
	with 2580648 stored elements in Compressed Sparse Row format>

In [18]:
# get_similiraty_items return top 5 NN
def get_similiraty_items(X_tfidf, item_id, topn=5):

    query = X_tfidf[item_id]
    scores = X_tfidf.dot(query.T).toarray().ravel()
    best = np.argpartition(scores, -topn)[-topn:] # return index!
    return sorted(zip(best, scores[best]), key=lambda x: -x[1])

In [19]:
similar_items = get_similiraty_items(X_tfidf, item_id=15)

In [22]:
for similar_item, similariti in similar_items:
    print(similar_item, similariti)
    item_description = df.loc[similar_item]['text']
    print('similar item id:', similar_item)
    print('similarity:', similariti)
    print('item description:', item_description)
    print()

15 1.0000000000000002
similar item id: 15
similarity: 1.0000000000000002
item description: is cleaning the house for her family who is comming later today..

10072 0.5336534905823506
similar item id: 10072
similarity: 0.5336534905823506
item description: Cleaning the house

4495 0.48954643197016023
similar item id: 4495
similarity: 0.48954643197016023
item description:  hot weather for the lose. And I have to clean too!  Cleaning in the heat sucks balls.

678 0.45118380147029297
similar item id: 678
similarity: 0.45118380147029297
item description: Searching my home for a few things to cook them for dinner this evening. It`s mothers day so guess who im eating with

3382 0.4492543819216047
similar item id: 3382
similarity: 0.4492543819216047
item description: cleaning cleaning cleaning today then working out!  i love not working!



In [23]:
def generate_random_vectofizer(dim, n_vectors):
    return np.random.randn(dim, n_vectors)

In [29]:
vocab_size = len(tfidf.get_feature_names())
print('vocabulary size:', vocab_size)

np.random.seed(0)
n_vectors = 16
random_vectors = generate_random_vectofizer(vocab_size, n_vectors)
print('dimension:', random_vectors.shape)

random_vectors

vocabulary size: 24002
dimension: (24002, 16)


/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


array([[ 1.76405235,  0.40015721,  0.97873798, ...,  0.12167502,
         0.44386323,  0.33367433],
       [ 1.49407907, -0.20515826,  0.3130677 , ...,  1.46935877,
         0.15494743,  0.37816252],
       [-0.88778575, -1.98079647, -0.34791215, ..., -0.4380743 ,
        -1.25279536,  0.77749036],
       ...,
       [-1.31821475,  0.36817968,  0.53415198, ...,  1.60818267,
         0.57866133, -1.7089848 ],
       [-0.83863877,  0.61162805,  0.83759737, ...,  0.8996262 ,
         0.04076996, -2.66958722],
       [-0.28932216,  0.7880676 , -0.01545824, ..., -1.54590442,
         0.54839386,  0.87119482]])

In [31]:
data_point = X_tfidf[0]

bin_indices_bits = data_point.dot(random_vectors) >= 0

print('dimension:', bin_indices_bits.shape)
bin_indices_bits

dimension: (1, 16)


array([[ True,  True, False,  True, False, False,  True,  True, False,
        False, False, False,  True,  True,  True, False]])

In [34]:
powers_two = 1 << np.arange(n_vectors-1, -1, step=-1)
print(powers_two)

bin_indices = bin_indices_bits.dot(powers_two)
print(bin_indices)

[32768 16384  8192  4096  2048  1024   512   256   128    64    32    16
     8     4     2     1]
[54030]


In [38]:
bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
print(bin_indices_bits.shape)

bin_indices = bin_indices_bits.dot(powers_two)
bin_indices.shape

(21027, 16)


(21027,)

In [39]:
from collections import defaultdict

In [63]:
def train_lsh(X_tfidf, n_vectors, seed=None):
    if seed is not None:
        np.random.seed(seed)

    dim = X_tfidf.shape[1]
    random_vectors = generate_random_vectofizer(dim, n_vectors)

    bin_indices_bits = X_tfidf.dot(random_vectors) >= 0
    power_of_two = 1 << np.arange(n_vectors-1, -1, step=-1)
    bin_indices = bin_indices_bits.dot(power_of_two)

    table = defaultdict(list)
    for idx, bin_index in enumerate(bin_indices):
        table[bin_index].append(idx)

    model={
        'table': table,
        'random_vectors': random_vectors,
        'bin_indices': bin_indices,
        'bin_indices_bits': bin_indices_bits
    }
    return model

n_vectors = 16
model = train_lsh(X_tfidf, n_vectors, seed=143)    

In [64]:
similar_items_ids = [similar_item for similar_item, _ in similar_items]
bits1 = model['bin_indices_bits'][similar_items_ids[0]]
bits2 = model['bin_indices_bits'][similar_items_ids[1]]

print('bits 1:', bits1)
print('bits 2:', bits2)
print('Number of agreed bins:', np.sum(bits1 == bits2))

bits 1: [False False  True False  True  True False False False  True  True False
 False False False False]
bits 2: [False False  True False  True  True  True False False  True False False
 False  True False False]
Number of agreed bins: 13


In [44]:
from itertools import combinations

In [65]:
def search_nearby_bins(query_bin_bits, table, search_radius=3, candidate_set=None):
    if candidate_set is None:
        candidate_set = set()

    n_vectors = query_bin_bits.shape[0]
    powers_of_two = 1 << np.arange(n_vectors-1,-1, step=-1)

    for different_bits in combinations(range(n_vectors), search_radius):
        index = list(different_bits)
        alternate_bits = query_bin_bits.copy()
        alternate_bits[index] = np.logical_not(alternate_bits[index])

        nearby_bin = alternate_bits.dot(powers_two)

        if nearby_bin in table:
            candidate_set.update(table[nearby_bin])

    return candidate_set

In [74]:
from sklearn.metrics.pairwise import pairwise_distances


In [75]:
def get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=3):
    table = model['table']
    random_vectors = model['random_vectors']

    # compute bin index for the query vector, in bit representation.
    bin_index_bits = np.ravel(query_vector.dot(random_vectors) >= 0)

    # search nearby bins and collect candidates
    candidate_set = set()
    for search_radius in range(max_search_radius + 1):
        candidate_set = search_nearby_bins(bin_index_bits, table, search_radius, candidate_set)

    # sort candidates by their true distances from the query
    candidate_list = list(candidate_set)
    candidates = X_tfidf[candidate_list]
    distance = pairwise_distances(candidates, query_vector, metric='cosine').flatten()
    
    distance_col = 'distance'
    nearest_neighbors = pd.DataFrame({
        'id': candidate_list, distance_col: distance
    }).sort_values(distance_col).reset_index(drop=True)
    return nearest_neighbors

In [76]:
print('original similar items:\n' + str(similar_items))

item_id = 15
query_vector = X_tfidf[item_id]
nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=5)
print('dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

original similar items:
[(15, 1.0000000000000002), (10072, 0.5336534905823506), (4495, 0.48954643197016023), (678, 0.45118380147029297), (3382, 0.4492543819216047)]
dimension:  (3292, 2)


,id,distance
0,15,0.000000
1,10072,0.466347
2,4495,0.510454
3,3382,0.550746
4,12193,0.552188


In [77]:
nearest_neighbors = get_nearest_neighbors(X_tfidf, query_vector, model, max_search_radius=10)
print('dimension: ', nearest_neighbors.shape)
nearest_neighbors.head()

dimension:  (20087, 2)


,id,distance
0,15,0.000000
1,10072,0.466347
2,4495,0.510454
3,678,0.548816
4,3382,0.550746


In [78]:

# we can perform a join with the original table to get the description
# for sanity checking purpose
nearest_neighbors.head().merge(df, on='id', how='inner')

,id,distance,text
0,15,0.000000,is cleaning the house for her family who is co...
1,10072,0.466347,Cleaning the house
2,4495,0.510454,hot weather for the lose. And I have to clean...
3,678,0.548816,Searching my home for a few things to cook the...
4,3382,0.550746,cleaning cleaning cleaning today then working ...
